# Import a GF Instruments CMD csv file, convert it to geojson and plot on a map

O. Kaufmann, 2018

In [3]:
import os
import pandas as pd
import folium
import geojson
import json
import numpy as np
from osgeo import ogr # osgeo is in pygdal package

In [8]:
pwd

'/home/su530201/PycharmProjects/stage-geophy/donnees_fournies/exemples_notebooks_python'

In [13]:
home_dir = '../sample_data/'
#filename = '/home/su530201/PycharmProjects/stage-geophy/traitement/example_notebooks/sample_data/'
#filename = 'CMD_UTM31N.dat'
filename = 'CMD_WGS84.dat'
zoom_start = 15

###  ** If geographic coordinates are given, WGS84 crs is assumed otherwise EPSG:32631 is assumed**

In [14]:
input_crs_epsg = 32631  # WGS84 UTM 31N
output_crs_epsg = 4326  # WGS84

in_crs = ogr.osr.SpatialReference()
in_crs.ImportFromEPSG(input_crs_epsg)

out_crs = ogr.osr.SpatialReference()
out_crs.ImportFromEPSG(output_crs_epsg)

crs_transform = ogr.osr.CoordinateTransformation(in_crs, out_crs)

### Convert degrees minutes to decimal degrees

In [15]:
def ddmm_to_dd(x):
    degrees = float(x) // 100
    minutes = x - 100 * degrees
    return degrees + minutes/60

In [16]:
df= pd.read_csv(home_dir + filename, sep='\t')

FileNotFoundError: [Errno 2] File ../sample_data/CMD_WGS84.dat does not exist: '../sample_data/CMD_WGS84.dat'

In [17]:
if ('Latitude' in df.columns) and ('Longitude' in df.columns):
    df['Latitude']=pd.to_numeric(df['Latitude'].str.rstrip('N'), errors='ignore').map(lambda x: ddmm_to_dd(x))
    df['Longitude']=pd.to_numeric(df['Longitude'].str.rstrip('E'), errors='ignore').map(lambda x: ddmm_to_dd(x))
    WGS84_input = True
elif ('Easting' in df.columns) and ('Northing' in df.columns):
    WGS84_input = False
else:
    WGS84_input = None
    print('### Unexpected dataframe format! ###')

NameError: name 'df' is not defined

In [48]:
df

,Latitude,Longitude,Altitude,Time,Cond.1[mS/m],Inph.1[ppt],Cond.2[mS/m],Inph.2[ppt],Cond.3[mS/m],Inph.3[ppt],Inv.Cond.1[mS/m],Inv.Cond.2[mS/m],Inv.Thick [m],Inv.RMS[%],Note
0,50.140167,5.181265,164.5,07:43:55.36,13.11,1.70,11.31,4.33,10.60,5.87,23.76,7.00,0.7,4.0,NaN
1,50.140168,5.181267,164.5,07:43:55.77,13.09,1.69,11.27,4.36,10.42,5.73,23.36,6.89,0.7,3.1,NaN
2,50.140170,5.181270,164.5,07:43:56.36,13.50,1.69,11.48,4.34,10.78,5.63,24.17,7.12,0.7,3.6,NaN
3,50.140170,5.181270,164.5,07:43:56.81,13.55,1.69,11.46,4.36,10.44,5.40,23.41,6.90,0.7,3.6,NaN
4,50.140183,5.181270,164.9,07:43:57.43,13.78,1.69,11.59,4.30,10.24,5.04,57.43,6.69,0.2,3.9,NaN
5,50.140183,5.181270,164.9,07:43:57.95,13.74,1.68,11.49,4.22,10.38,5.07,23.28,6.86,0.7,4.0,NaN
6,50.140192,5.181270,164.8,07:43:58.29,13.59,1.69,11.41,4.24,10.20,4.84,17.94,5.18,1.4,2.9,NaN
7,50.140192,5.181270,164.8,07:43:58.81,13.40,1.68,11.62,4.23,10.56,4.74,23.69,6.98,0.7,2.7,NaN
8,50.140193,5.181273,165.1,07:43:59.40,13.35,1.68,11.27,4.22,10.29,4.31,23.08,6.80,0.7,3.8,NaN
9,50.140193,5.181273,165.1,07:43:59.85,13.46,1.67,11.19,4.06,10.13,4.13,17.82,5.14,1.4,3.2,NaN


### Filter dataframe: remove rows without position

In [49]:
if WGS84_input:
    df = df[df['Latitude'].notnull()]
elif not WGS84_input:
    df = df[df['Northing'].notnull()]
    df.loc[:,'Latitude'] = pd.Series(np.nan, index=df.index)
    df.loc[:,'Longitude'] = pd.Series(np.nan, index=df.index)

## Reproject if needed and save data to a geojson file 
### *TODO: Add properties to the geojson file...*

In [50]:
stations = []

In [51]:
for idx, row in df.iterrows():
    # properties = {}
    # for col in df.columns:
    #    properties[col]=row[col]
    # print(properties)
    if WGS84_input:
        station_location = geojson.Point((row['Longitude'], row['Latitude'], row['Altitude']))
    if not WGS84_input:
        station_location = geojson.Point((row['Easting'], row['Northing'], row['Altitude']))
        point = ogr.CreateGeometryFromJson(str(station_location))
        point.Transform(crs_transform)
        station_location = point.ExportToJson()
        station_location = geojson.loads(station_location)
        df.set_value(idx, 'Latitude', station_location['coordinates'][1])
        df.set_value(idx, 'Longitude', station_location['coordinates'][0])
    station = geojson.Feature(geometry=station_location, properties={'id': str(idx), 'time': str(row['Time']), 'C1': row['Cond.1[mS/m]']})
    stations.append(station)
stations_geojson = geojson.FeatureCollection(stations)

In [52]:
stations_geojson

{"features": [{"geometry": {"coordinates": [5.181265, 50.140166666666666, 164.5], "type": "Point"}, "properties": {"C1": 13.11, "id": "0", "time": "07:43:55.36"}, "type": "Feature"}, {"geometry": {"coordinates": [5.181266666666667, 50.140168333333335, 164.5], "type": "Point"}, "properties": {"C1": 13.09, "id": "1", "time": "07:43:55.77"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.140170000000005, 164.5], "type": "Point"}, "properties": {"C1": 13.5, "id": "2", "time": "07:43:56.36"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.140170000000005, 164.5], "type": "Point"}, "properties": {"C1": 13.55, "id": "3", "time": "07:43:56.81"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.14018333333333, 164.9], "type": "Point"}, "properties": {"C1": 13.78, "id": "4", "time": "07:43:57.43"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.14018333333333, 164.9], "type": "Point"}, "properties": {"C1": 13.74, "id": "5", "time": "07:43

### Save stations as a geojson file

In [53]:
jsn_filename = home_dir + os.path.splitext(os.path.basename(filename))[0] + '.jsn'
jsn_filename

'/home/su530201/PycharmProjects/stage-geophy/donnees_fournies/exemples_notebooks_python/sample_data/CMD_WGS84.jsn'

In [54]:
with open(jsn_filename, 'w', encoding='utf-8') as f:
    f.write(geojson.dumps(stations_geojson))

In [55]:
stations_geojson

{"features": [{"geometry": {"coordinates": [5.181265, 50.140166666666666, 164.5], "type": "Point"}, "properties": {"C1": 13.11, "id": "0", "time": "07:43:55.36"}, "type": "Feature"}, {"geometry": {"coordinates": [5.181266666666667, 50.140168333333335, 164.5], "type": "Point"}, "properties": {"C1": 13.09, "id": "1", "time": "07:43:55.77"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.140170000000005, 164.5], "type": "Point"}, "properties": {"C1": 13.5, "id": "2", "time": "07:43:56.36"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.140170000000005, 164.5], "type": "Point"}, "properties": {"C1": 13.55, "id": "3", "time": "07:43:56.81"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.14018333333333, 164.9], "type": "Point"}, "properties": {"C1": 13.78, "id": "4", "time": "07:43:57.43"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.14018333333333, 164.9], "type": "Point"}, "properties": {"C1": 13.74, "id": "5", "time": "07:43

### Flip coordinates in geojson 
#### This is needed with geographic coordinates because leaflet uses the Latitude, Longitude, Altitude order while geojson format is Longitude, Latitude, Altitude

In [56]:
def flip_geojson_coordinates(geo):
    if isinstance(geo, dict):
        for k, v in geo.items():
            if k == 'coordinates':
                z = np.array(geo[k])
                f = z.flatten()
                geo[k] = np.dstack((f[1::2], f[::2])).reshape(z.shape).tolist()
            else:
                print(k,v)
    elif isinstance(geo, list):
        for k in geo:
            flip_geojson_coordinates(k)

In [57]:
flip_geojson_coordinates(stations_geojson)
stations_geojson

features [{"geometry": {"coordinates": [5.181265, 50.140166666666666, 164.5], "type": "Point"}, "properties": {"C1": 13.11, "id": "0", "time": "07:43:55.36"}, "type": "Feature"}, {"geometry": {"coordinates": [5.181266666666667, 50.140168333333335, 164.5], "type": "Point"}, "properties": {"C1": 13.09, "id": "1", "time": "07:43:55.77"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.140170000000005, 164.5], "type": "Point"}, "properties": {"C1": 13.5, "id": "2", "time": "07:43:56.36"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.140170000000005, 164.5], "type": "Point"}, "properties": {"C1": 13.55, "id": "3", "time": "07:43:56.81"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.14018333333333, 164.9], "type": "Point"}, "properties": {"C1": 13.78, "id": "4", "time": "07:43:57.43"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.14018333333333, 164.9], "type": "Point"}, "properties": {"C1": 13.74, "id": "5", "time": "07:43:57.

{"features": [{"geometry": {"coordinates": [5.181265, 50.140166666666666, 164.5], "type": "Point"}, "properties": {"C1": 13.11, "id": "0", "time": "07:43:55.36"}, "type": "Feature"}, {"geometry": {"coordinates": [5.181266666666667, 50.140168333333335, 164.5], "type": "Point"}, "properties": {"C1": 13.09, "id": "1", "time": "07:43:55.77"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.140170000000005, 164.5], "type": "Point"}, "properties": {"C1": 13.5, "id": "2", "time": "07:43:56.36"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.140170000000005, 164.5], "type": "Point"}, "properties": {"C1": 13.55, "id": "3", "time": "07:43:56.81"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.14018333333333, 164.9], "type": "Point"}, "properties": {"C1": 13.78, "id": "4", "time": "07:43:57.43"}, "type": "Feature"}, {"geometry": {"coordinates": [5.18127, 50.14018333333333, 164.9], "type": "Point"}, "properties": {"C1": 13.74, "id": "5", "time": "07:43

## Plot the map

### Define the basemap and add markers from positions store

In [58]:
center = [np.mean([i['geometry']['coordinates'][0] for i in stations_geojson.features]), np.mean([i['geometry']['coordinates'][1] for i in stations_geojson.features])]
print(center)
map_osm = folium.Map(location=center, zoom_start=zoom_start, control_scale=True)
for i in stations_geojson.features:
    #print(i)
    folium.CircleMarker(location=[i['geometry']['coordinates'][0], i['geometry']['coordinates'][1]], popup=str(i['properties']['id']), radius=0.75, color='#dd2222', fill_color='#dd2222').add_to(map_osm)

[5.182611509598604, 50.13900993891798]


### Import a geojson file

In [59]:
with open(home_dir + '2017-03-26_08-50_Sun.geojson') as f:
    geojson_dataset =  json.load(f)
# flip_geojson_coordinates(geojson_dataset)
folium.GeoJson(geojson_dataset,name='geojson').add_to(map_osm)

### Display map

In [60]:
map_osm

In [61]:
import branca

In [62]:
field = 'Cond.1[mS/m]'
min_val = df[field].min()
max_val = df[field].max()

In [63]:
cm = branca.colormap.LinearColormap(['green', 'yellow', 'red'], vmin=min_val, vmax=max_val, caption=field )

In [64]:
f = folium.map.FeatureGroup()

for idx, row in df.iterrows():
    f.add_child(
        folium.CircleMarker(location=[row['Latitude'], row['Longitude']], 
                            popup='hello', 
                            radius=row[field],
                            color=None, 
                            fill_color=cm(row[field]),
                            fill_opacity=0.6)
    )

In [65]:
m = folium.Map(location=center, zoom_start=zoom_start, control_scale=True)
m.add_child(f)
m.add_child(cm)

In [24]:
cm